In [1]:
import torch
print(torch.__version__)  # Should print 2.x.x
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Should print GTX 1650 Ti

2.6.0+cu118
True
NVIDIA GeForce GTX 1650 Ti


In [2]:
import os
from pathlib import Path

In [3]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation'

In [6]:
ls

config/       dvc.yaml  logs/        requirements.txt  src/
Data/         github/   params.yaml  research/         template.py
Data_Zip.zip  LICENSE   README.md    setup.py          templates/


In [12]:
!pip install gdown

In [ ]:
# #for the folder 
# import gdown 

# url = 'https://drive.google.com/drive/folders/19aBmKV2W7iwY0MzPjVmEM4a-dDxFvqOM?usp=sharing'
# output_folder = 'Data'


# gdown.download_folder(url , output=output_folder , use_cookies=False , quiet=False)

Retrieving folder contents


Processing file 1bOgu_rWcJqBRdmHqKzvD98QplpURHaIn Heart.zip


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1bOgu_rWcJqBRdmHqKzvD98QplpURHaIn
From (redirected): https://drive.google.com/uc?id=1bOgu_rWcJqBRdmHqKzvD98QplpURHaIn&confirm=t&uuid=2804f532-5765-4869-b2c6-0d97cddddf5c
To: /home/priyanshu1303d/Projects/Heart_Segmentation/Data/Heart.zip
100%|██████████| 454M/454M [00:12<00:00, 37.6MB/s] 
Download completed


['Data/Heart.zip']

In [36]:
### for zip files downlaod
# import gdown 
# import zipfile 

# zip_url = 'https://drive.google.com/file/d/1uPCqKVlKLHoW52oLBvdw0hSQRVmMLfML/view?usp=sharing'
# zip_path = 'Data/data.zip' # the folder must be 

# gdown.download(zip_url , zip_path , quiet = False)

In [ ]:
# #Extracting the folder 
# import zipfile
# with zipfile.ZipFile('Data/Heart.zip', 'r') as zip_ref:
#     zip_ref.extractall("Data/data_ingetsion")

In [41]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation'

In [ ]:
# from ensure import ensure_annotations

# @ensure_annotations
# def zip_extract(zip_path: str, dest_path: str):
#     '''
#     Extracts the contents of a zip file to a specified destination directory.

#     Args:
#         zip_path (str): Path to the zip file to be extracted.
#         dest_path (str): Directory where the zip contents will be extracted.

#     Raises:
#         FileNotFoundError: If the zip file does not exist.
#         zipfile.BadZipFile: If the file is not a valid zip archive.
#         PermissionError: If the destination directory is not writable.
#     '''
#     if not os.path.exists(zip_path):
#         raise FileNotFoundError(f"Zip file not found at: {zip_path}")
    
#     try:
#         with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#             zip_ref.extractall(dest_path)
#     except zipfile.BadZipFile:
#         raise zipfile.BadZipFile(f"{zip_path} is not a valid zip file")
#     except PermissionError:
#         raise PermissionError(f"Cannot write to destination: {dest_path}")

In [ ]:
# dest_path = 'Data/data_ingestion'
# zip_path = 'Data/Heart.zip'

# zip_extract(zip_path , dest_path)

# Modular Coding

In [7]:
%pwd

'/home/priyanshu1303d/Projects/Heart_Segmentation'

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_path  : Path
    unzip_path : Path

In [22]:
from src.Heart_Segmentation.constants import *
from src.Heart_Segmentation.utils.common import read_yaml ,create_directories

In [23]:
class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingetsion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_path= config.local_data_path,
            unzip_path= config.unzip_path
        )

        return data_ingestion_config

In [24]:
from pathlib import Path
import os
import zipfile
from src.Heart_Segmentation import logger
from src.Heart_Segmentation.utils.common import get_size
import gdown

In [27]:
class DataIngestion:
    def __init__(self , config : DataIngestionConfig):
        self.config = config


    def download_dataset(self):
        source_url = self.config.source_url
        unzip_path = self.config.unzip_path

        if not os.path.exists(self.config.local_data_path):
            gdown.download_folder(url = source_url , output= unzip_path, use_cookies=False , quiet= False)
            logger.info(f"Dataset downloaded!!")
        else:
            logger.info(f"file already exists of size : {get_size(Path(self.config.local_data_file))}")

    def zip_extract(self):
        local_data_path = self.config.local_data_path
        unzip_path = self.config.unzip_path

        with zipfile.ZipFile(local_data_path , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"File successfully extracted")

In [28]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingetsion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_dataset()
    data_ingestion.zip_extract()
except Exception as e:
    raise e

[2025-03-26 10:23:37,234: INFO: common : yaml file: config/config.yaml loaded successfully]
[2025-03-26 10:23:37,237: INFO: common : yaml file: params.yaml loaded successfully]
[2025-03-26 10:23:37,238: INFO: common : created directory at: artifacts]
[2025-03-26 10:23:37,239: INFO: common : created directory at: artifacts/data_ingestion]


Retrieving folder contents


Processing file 1bOgu_rWcJqBRdmHqKzvD98QplpURHaIn Heart.zip


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1bOgu_rWcJqBRdmHqKzvD98QplpURHaIn
From (redirected): https://drive.google.com/uc?id=1bOgu_rWcJqBRdmHqKzvD98QplpURHaIn&confirm=t&uuid=22999908-0a80-4ff9-8fa6-5630a3d4ab1e
To: /home/priyanshu1303d/Projects/Heart_Segmentation/artifacts/data_ingestion/Heart.zip
100%|██████████| 454M/454M [00:11<00:00, 39.6MB/s] 

[2025-03-26 10:23:55,535: INFO: 1297925953 : Dataset downloaded!!]



Download completed


[2025-03-26 10:23:57,883: INFO: 1297925953 : File successfully extracted]


# Data Ingestion Completed